In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import pywt

In [2]:
df = pd.read_excel("df_50.xlsx")
df.head()

,AvgAir_T_TempC,MaxAir_T_TempC,MinAir_T_TempC,Avg_Soil_TP50_TempC,Max_Soil_TP50_TempC,Min_Soil_TP50_TempC
0,24.24,30.42,0.00,15.92,14.07,12.49
1,22.25,28.47,16.63,13.03,14.07,12.49
2,20.45,24.77,16.77,12.35,12.49,12.28
3,20.11,25.32,14.59,12.28,12.30,12.26
4,23.52,28.47,17.78,12.31,12.34,12.28


In [3]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 433 entries, 0 to 432
Data columns (total 6 columns):
 #   Column               Non-Null Count  Dtype  
---  ------               --------------  -----  
 0   AvgAir_T_TempC       433 non-null    float64
 1   MaxAir_T_TempC       433 non-null    float64
 2   MinAir_T_TempC       433 non-null    float64
 3   Avg_Soil_TP50_TempC  433 non-null    float64
 4   Max_Soil_TP50_TempC  433 non-null    float64
 5   Min_Soil_TP50_TempC  433 non-null    float64
dtypes: float64(6)
memory usage: 20.4 KB


In [4]:
df["Avg_Soil_TP50_TempC_Plus_1"] = df["Avg_Soil_TP50_TempC"].shift(-1)
df["Avg_Soil_TP50_TempC_Plus_2"] = df["Avg_Soil_TP50_TempC"].shift(-2)

df["Max_Soil_TP50_TempC_Plus_1"] = df["Max_Soil_TP50_TempC"].shift(-1)
df["Max_Soil_TP50_TempC_Plus_2"] = df["Max_Soil_TP50_TempC"].shift(-2)

df["Min_Soil_TP50_TempC_Plus_1"] = df["Min_Soil_TP50_TempC"].shift(-1)
df["Min_Soil_TP50_TempC_Plus_2"] = df["Min_Soil_TP50_TempC"].shift(-2)

df["AvgAir_T_TempC_Minus_6"] = df["AvgAir_T_TempC"].shift(6)
df["AvgAir_T_TempC_Minus_5"] = df["AvgAir_T_TempC"].shift(5)
df["AvgAir_T_TempC_Minus_4"] = df["AvgAir_T_TempC"].shift(4)
df["AvgAir_T_TempC_Minus_3"] = df["AvgAir_T_TempC"].shift(3)
df["AvgAir_T_TempC_Minus_2"] = df["AvgAir_T_TempC"].shift(2)
df["AvgAir_T_TempC_Minus_1"] = df["AvgAir_T_TempC"].shift(1)

df["MaxAir_T_TempC_Minus_6"] = df["MaxAir_T_TempC"].shift(6)
df["MaxAir_T_TempC_Minus_5"] = df["MaxAir_T_TempC"].shift(5)
df["MaxAir_T_TempC_Minus_4"] = df["MaxAir_T_TempC"].shift(4)
df["MaxAir_T_TempC_Minus_3"] = df["MaxAir_T_TempC"].shift(3)
df["MaxAir_T_TempC_Minus_2"] = df["MaxAir_T_TempC"].shift(2)
df["MaxAir_T_TempC_Minus_1"] = df["MaxAir_T_TempC"].shift(1)


df["MinAir_T_TempC_Minus_6"] = df["MinAir_T_TempC"].shift(6)
df["MinAir_T_TempC_Minus_5"] = df["MinAir_T_TempC"].shift(5)
df["MinAir_T_TempC_Minus_4"] = df["MinAir_T_TempC"].shift(4)
df["MinAir_T_TempC_Minus_3"] = df["MinAir_T_TempC"].shift(3)
df["MinAir_T_TempC_Minus_2"] = df["MinAir_T_TempC"].shift(2)
df["MinAir_T_TempC_Minus_1"] = df["MinAir_T_TempC"].shift(1)

In [5]:
df = df[["AvgAir_T_TempC_Minus_6","MaxAir_T_TempC_Minus_6","MinAir_T_TempC_Minus_6",
          "AvgAir_T_TempC_Minus_5","MaxAir_T_TempC_Minus_5","MinAir_T_TempC_Minus_5",
          "AvgAir_T_TempC_Minus_4","MaxAir_T_TempC_Minus_4","MinAir_T_TempC_Minus_4",
          "AvgAir_T_TempC_Minus_3","MaxAir_T_TempC_Minus_3","MinAir_T_TempC_Minus_3",
          "AvgAir_T_TempC_Minus_2","MaxAir_T_TempC_Minus_2","MinAir_T_TempC_Minus_2",
          "AvgAir_T_TempC_Minus_1","MaxAir_T_TempC_Minus_1","MinAir_T_TempC_Minus_1",
          "AvgAir_T_TempC","MaxAir_T_TempC","MinAir_T_TempC",
          "Avg_Soil_TP50_TempC","Max_Soil_TP50_TempC","Min_Soil_TP50_TempC",
          "Avg_Soil_TP50_TempC_Plus_1","Max_Soil_TP50_TempC_Plus_1","Min_Soil_TP50_TempC_Plus_1",
          "Avg_Soil_TP50_TempC_Plus_2","Max_Soil_TP50_TempC_Plus_2","Min_Soil_TP50_TempC_Plus_2"]]
df.dropna(inplace=True)

In [6]:
df.reset_index(drop="True", inplace=True)

In [7]:
def WaveletTransform(x,walvelettype,level):
    
    size = np.shape(x)
    length = (size[0] // (2**level)) * (2 ** level)
    x = x.iloc[0:length]
    bigmats = []
    for i in x.columns:
        vec = x[i].values
        cAs_cDs = pywt.swt(vec, wavelet=walvelettype, level=level,axis=0) 
        for j in cAs_cDs:
            bigmats.append(j[0])
            bigmats.append(j[1])

    
    features_num = len(bigmats)
    x = np.zeros((length, features_num))
    for i in range(0 ,len(bigmats)):
        for j in range(0,len(bigmats[0])):
            x[j,i] = x[j,i] + bigmats[i][j]
    
    return(x)

In [8]:
x = df.drop(["Avg_Soil_TP50_TempC","Max_Soil_TP50_TempC","Min_Soil_TP50_TempC",
            "Avg_Soil_TP50_TempC_Plus_1","Max_Soil_TP50_TempC_Plus_1","Min_Soil_TP50_TempC_Plus_1",
            "Avg_Soil_TP50_TempC_Plus_2","Max_Soil_TP50_TempC_Plus_2","Min_Soil_TP50_TempC_Plus_2"],axis=1)
x

,AvgAir_T_TempC_Minus_6,MaxAir_T_TempC_Minus_6,MinAir_T_TempC_Minus_6,AvgAir_T_TempC_Minus_5,MaxAir_T_TempC_Minus_5,MinAir_T_TempC_Minus_5,AvgAir_T_TempC_Minus_4,MaxAir_T_TempC_Minus_4,MinAir_T_TempC_Minus_4,AvgAir_T_TempC_Minus_3,...,MinAir_T_TempC_Minus_3,AvgAir_T_TempC_Minus_2,MaxAir_T_TempC_Minus_2,MinAir_T_TempC_Minus_2,AvgAir_T_TempC_Minus_1,MaxAir_T_TempC_Minus_1,MinAir_T_TempC_Minus_1,AvgAir_T_TempC,MaxAir_T_TempC,MinAir_T_TempC
0,24.24,30.42,0.00,22.25,28.47,16.63,20.45,24.77,16.77,20.11,...,14.59,23.52,28.47,17.78,22.08,27.30,14.37,21.49,29.56,11.12
1,22.25,28.47,16.63,20.45,24.77,16.77,20.11,25.32,14.59,23.52,...,17.78,22.08,27.30,14.37,21.49,29.56,11.12,21.90,26.27,15.37
2,20.45,24.77,16.77,20.11,25.32,14.59,23.52,28.47,17.78,22.08,...,14.37,21.49,29.56,11.12,21.90,26.27,15.37,20.36,23.38,14.90
3,20.11,25.32,14.59,23.52,28.47,17.78,22.08,27.30,14.37,21.49,...,11.12,21.90,26.27,15.37,20.36,23.38,14.90,18.31,25.46,12.12
4,23.52,28.47,17.78,22.08,27.30,14.37,21.49,29.56,11.12,21.90,...,15.37,20.36,23.38,14.90,18.31,25.46,12.12,19.04,25.67,12.12
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
420,19.75,29.30,9.16,18.19,24.56,10.54,15.45,24.52,4.61,19.27,...,9.93,16.55,21.89,10.94,14.50,20.54,6.70,15.02,23.63,5.25
421,18.19,24.56,10.54,15.45,24.52,4.61,19.27,26.70,9.93,16.55,...,10.94,14.50,20.54,6.70,15.02,23.63,5.25,18.12,25.20,9.71
422,15.45,24.52,4.61,19.27,26.70,9.93,16.55,21.89,10.94,14.50,...,6.70,15.02,23.63,5.25,18.12,25.20,9.71,22.42,28.65,16.47
423,19.27,26.70,9.93,16.55,21.89,10.94,14.50,20.54,6.70,15.02,...,5.25,18.12,25.20,9.71,22.42,28.65,16.47,18.28,21.68,15.88


In [9]:
y = df[["Avg_Soil_TP50_TempC","Max_Soil_TP50_TempC","Min_Soil_TP50_TempC",
            "Avg_Soil_TP50_TempC_Plus_1","Max_Soil_TP50_TempC_Plus_1","Min_Soil_TP50_TempC_Plus_1",
            "Avg_Soil_TP50_TempC_Plus_2","Max_Soil_TP50_TempC_Plus_2","Min_Soil_TP50_TempC_Plus_2"]].values
y

array([[12.65, 12.77, 12.53, ..., 13.03, 13.2 , 12.81],
       [12.72, 13.03, 12.52, ..., 13.27, 13.33, 13.18],
       [13.03, 13.2 , 12.81, ..., 13.37, 13.4 , 13.32],
       ...,
       [14.2 , 14.24, 14.17, ..., 14.27, 14.3 , 14.24],
       [14.2 , 14.25, 14.16, ..., 14.31, 14.33, 14.28],
       [14.27, 14.3 , 14.24, ..., 14.27, 14.32, 14.18]])

In [10]:
x = WaveletTransform(x,"db5",5)

In [12]:
x.shape

(416, 210)